#Imports
import numpy as np 
import pandas as pd 
import FixedClassificationModel_JL as FixedClassificationModel
import Globals_JL as Globals
import CombineLigandsProteins_JL as CombineLigandsProteins

In [2]:
import Protein_Input_funcions as pif
import ReadingFasta_JL as ReadingFasta

In [26]:
import importlib 
importlib.reload(CombineLigandsProteins)
importlib.reload(FixedClassificationModel)

<module 'FixedClassificationModel_JL' from '/mnt/af3ff5c3-2943-4972-8c3a-6b98174779b7/Justice/Chaperone_Analysis/OR_ML/Rtp1s_RF/FixedClassificationModel_JL.py'>

In [ ]:
"""
The goal of this notebook is to test if we can build a RF model to predict 
weather an OR will be enhanced by Rtp1s or not 

input should simply be OR sequence with supervised enhance or not enhance 
"""

In [ ]:
"""
From here intend to adapt the pipeline built for aa_3di_smile + ligand 
for aa + rtp1s 
"""

In [5]:
TM_location = "./Temp_TM/TM.csv"
Di_location = "../data_files/3DiSequences/fullset_ss.fasta"
smile_location = "../Ligands_withSMILE/ligand_SMILES.csv"
experimental_results = "../input_results/"
accession_to_ensemble = "../ensemble_to_accession.csv"

OR_Rtp_dependency = pd.read_csv('./OR_RTP_dependency.csv')
sup_out = OR_Rtp_dependency[['accession number', 'enhance']]

In [27]:
# Generating matix for training 
matrix = CombineLigandsProteins.features_matrix(TM_location, 
                                                Di_location, 
                                                accession_to_ensemble, 
                                                sup_out)

Pos Observations: 24
Neg Observations: 20


In [28]:
testX = matrix['X']
testY = matrix['Y']
BALANCE = matrix['balance']

In [29]:
accuracy = 0
recall = 0
BAC = 0

a_TN = 0
a_FN = 0
a_TP = 0
a_FP = 0

loss = 0
for i in range(50):
    acc, rec, bac, TN, FN,\
    TP, FP, log_loss, clf = FixedClassificationModel.train(testX, testY, BALANCE)
    accuracy += acc
    recall += rec
    BAC += bac
    loss += log_loss
    
    print("run " + str(i) + " accuracy: " + str(acc))

    a_TN += TN
    a_FN += FN
    a_TP += TP
    a_FP += FP
print('Average Accuracy: ' + str(accuracy/50))
print('Average Recall: ' + str(recall/50))
print('Average Balanced: ' + str(BAC/50))

print('Average TN: ' + str(a_TN/50))
print('Average FN: ' + str(a_FN/50))
print('Average TP: ' + str(a_TP/50))
print('Average FP: ' + str(a_FP/50))

print('Average Log Loss: ' + str(loss/50))

run 0 accuracy: 0.5
run 1 accuracy: 0.5104166666666666
run 2 accuracy: 0.44791666666666663
run 3 accuracy: 0.4270833333333333
run 4 accuracy: 0.5520833333333334
run 5 accuracy: 0.43749999999999994
run 6 accuracy: 0.5
run 7 accuracy: 0.6222222222222222
run 8 accuracy: 0.5729166666666667
run 9 accuracy: 0.5888888888888889
run 10 accuracy: 0.5888888888888889
run 11 accuracy: 0.40625
run 12 accuracy: 0.29591836734693877
run 13 accuracy: 0.4375
run 14 accuracy: 0.5111111111111111
run 15 accuracy: 0.41836734693877553
run 16 accuracy: 0.5104166666666667
run 17 accuracy: 0.46875
run 18 accuracy: 0.5510204081632653
run 19 accuracy: 0.21111111111111108
run 20 accuracy: 0.7708333333333334
run 21 accuracy: 0.5777777777777777
run 22 accuracy: 0.5520833333333334
run 23 accuracy: 0.42222222222222217
run 24 accuracy: 0.34375
run 25 accuracy: 0.7755102040816326
run 26 accuracy: 0.5666666666666667
run 27 accuracy: 0.5888888888888888
run 28 accuracy: 0.7333333333333334
run 29 accuracy: 0.4111111111111111

In [30]:
# Extract the important feature into a dataframe for visualization 
feat_importance = pd.DataFrame({'kmer':matrix['features'],
              'Importance':clf.feature_importances_}).sort_values('Importance', ascending=False)

TM = ['TM1', 'TM2', 'TM3', 'TM4', 'TM5', 'TM6', 'TM7']
for i in TM:
    feat_importance.loc[feat_importance['kmer'].str.contains(i), 'TM'] = i
feat_importance.loc[feat_importance['kmer'].str.contains('aa'), 'feature'] = 'aa'
feat_importance.loc[feat_importance['kmer'].str.contains('di'), 'feature'] = 'di'

In [183]:
feat_importance[['TM', 'Importance']].groupby('TM').sum()

,Importance
TM,
TM1,0.134105
TM2,0.145067
TM3,0.135123
TM4,0.148293
TM5,0.124829
TM6,0.169256
TM7,0.143327


In [184]:
feat_importance[['feature', 'Importance']].groupby('feature').sum()

,Importance
feature,
aa,0.621464
di,0.378536
